<a href="https://colab.research.google.com/github/pranjaldoshi96/MusicEmotionRecognition/blob/master/MusicEmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 29kB/s 
     |████████████████████████████████| 491kB 33.5MB/s 
     |████████████████████████████████| 3.2MB 32.3MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [5]:
pip install keras_self_attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=a534e9dbe0aa6c6f1f31c7141427336f77682adeaadf76ad63ce5f47f1ced1b3
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention


In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K
from keras.models import load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import TensorBoard
from keras_self_attention import SeqSelfAttention

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import pickle


In [0]:
def R2(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def Kendalltau(y_true,y_pred):
    return stats.kendalltau(y_true, y_pred,nan_policy='omit')

def build_model(input_shape,output_dim,return_sequences=True):
    model = Sequential()
    model.add(LSTM(input_shape=input_shape,output_dim=300,return_sequences=return_sequences))
    model.add(LSTM(input_shape=input_shape,output_dim=128,return_sequences=return_sequences))
    model.add(SeqSelfAttention(attention_activation='sigmoid',units=128))
    model.add(LSTM(input_dim=128,output_dim=1,return_sequences=True))
    model.add(Activation("linear"))
    model.compile(loss="mse", optimizer="rmsprop",metrics=[R2,'mae',Kendalltau])
    return model



In [0]:
def train_and_evaluate(model,X_train,y_train,X_test,y_test):
    checkpoint = ModelCheckpoint(filepath='./models/attention_valence_std_norm/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0006, patience=3, verbose=1,restore_best_weights=False)
    tensorboard = TensorBoard(log_dir='./all_logs/attention_valence_std_norm', write_graph=True)
    history = model.fit(X_train,
                y_train,
                batch_size=20,
                epochs=50,
                validation_split=0.05)#,callbacks=[checkpoint,tensorboard])
    result = model.evaluate(X_test,y_test)
    print("-----------------Result----------------")
    print(result)
    prediction = model.predict(X_test)
    corr_avg = 0
    for i in range(len(prediction)):
        oned_pred = prediction[i].flatten()
        oned_label = y_test[i].flatten()
        corr,pre = Kendalltau(oned_pred,oned_label)
        corr_avg += corr
    corr_avg = corr_avg / len(prediction)
    print("Kendall cofficient : ",corr_avg)


In [0]:
print('loading data ...')
with open('features.pkl', 'rb') as in_:
    feat = pickle.load(in_)
print('Data loaded... Shape : ',feat.shape)
features_without_time = feat[:,:,1:]

print("Reading arousal label")
arousal_label  = np.loadtxt('../annotations/arousal_cont_average.csv',delimiter=',',skiprows=1).astype(float)
# valence_label  = np.loadtxt('../annotations/valence_cont_average.csv',delimiter=',',skiprows=1).astype(float)

#Removing Song Label from data
arousal_label = arousal_label[:,1:]
# valence_label = valence_label[:,1:]

#Reshaping for Input to LSTM
arousal_label = arousal_label[:, :, None]
# valence_label = valence_label[:, :, None]

label_count = feat.shape[0]
sequence_len = feat.shape[1]
features_count = feat.shape[2]
feat = feat.reshape(label_count,features_count * sequence_len)
feat = StandardScaler().fit_transform(feat)
#feat = normalize(feat, axis=0, norm='max')
feat = feat.reshape(label_count,sequence_len,features_count)

#output = np.c_[arousal_label,valence_label]
#print(feat.shape,output.shape)

#Train Test split

train_data, test_data, train_label, test_label = train_test_split(
    feat, arousal_label, test_size=0.10)


In [0]:
label_count = train_data.shape[0]
sequence_len = train_data.shape[1]
features_count = train_data.shape[2]

input_shape = (sequence_len,features_count)
output_dim = 2
print(input_shape,output_dim)


In [0]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=2645312378)

train_data = train_data.reshape(train_data.shape[0],features_count * sequence_len)
train_label = train_label.reshape(train_label.shape[0],sequence_len)

sanga_counter = 1

for train_index, test_index in kfolds.split(train_data, train_label):

    print("\n\n ------------------------------------------------------  ", sanga_counter, "----------------------------------------------- \n\n")
    sanga_counter = sanga_counter + 1
    X_train, X_test = train_data[train_index], train_data[test_index]
    y_train, y_test = train_label[train_index], train_label[test_index]
    X_train, X_test = X_train.reshape(X_train.shape[0],sequence_len,features_count), X_test.reshape(X_test.shape[0],sequence_len,features_count)
    y_train, y_test = y_train[:, :, None], y_test[:, :, None]


    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    model = None
    model = build_model(input_shape,output_dim)
    train_and_evaluate(model,X_train,y_train,X_test,y_test)
    result = model.evaluate(test_data,test_label)
    print("-----------------Final Result----------------")
    print("---------------------------------------------")
    print(result)
    prediction = model.predict(test_data)
    corr_avg = 0
    for i in range(len(prediction)):
        oned_pred = prediction[i].flatten()
        oned_label = test_label[i].flatten()
        corr,pre = Kendalltau(oned_pred,oned_label)
        corr_avg += corr
    corr_avg = corr_avg / len(prediction)
    print("Kendall cofficient : ",corr_avg)
    print("---------------------------------------------")
    print("---------------------------------------------")